In [ ]:
adata_protein = sc.read_h5ad('../data/pbmc/pbmc_protein.h5ad')
adata_gene = sc.read_h5ad('../data/pbmc/pbmc_gene.h5ad')

In [ ]:
adata_malt_gene = sc.read_10x_h5("../Data/malt_10k_protein_v3_filtered_feature_bc_matrix.h5") # 8412,33538

In [ ]:
adata_malt = sc.read("../Data/filtered_feature_bc_matrix/matrix.mtx").T # 8412,33555
malt_features =  pd.read_csv("../Data/filtered_feature_bc_matrix/features.tsv", sep="\t", header=None)

In [ ]:
adata_malt.var["feature_type"] = list(malt_features[2])
adata_malt.obs_names = adata_malt_gene.obs_names
adata_malt.var['protein_names'] = list(malt_features[0])
adata_malt.var_names = list(malt_features[0])

In [ ]:
adata_malt_protein = adata_malt[:,adata_malt.var['feature_type'] == 'Antibody Capture'] # malt 蛋白质17 cells8412，proteins17

In [ ]:
adata_malt_gene.var_names_make_unique()

In [ ]:
adata_gene_test = adata_malt_gene.copy()

Hold out the protein expression for the MALT data

In [ ]:
adata_protein_test = adata_malt_protein.copy()
adata_protein_test.obs['sample'] = [1]*8412
adata_protein_test

Among the 17 proteins in the MALT dataset, 10 overlapped with the proteins in the PBMC data.

In [ ]:
ref = set(adata_protein_test.var.index)
prots = [] # 10
for x in adata_protein.var.index:
    if x in ref:
        prots.append(x)

prots

在两个数据集中测序的蛋白质中，MALT数据集中的平均蛋白质表达比PBMC数据集中的高四倍以上，表明了这两个数据集之间的内在差异

In [ ]:
adata_protein_test[:, prots].X.mean()/adata_protein[:, prots].X.mean()

In [ ]:
common_genes = np.intersect1d(adata_gene.var.index, adata_gene_test.var.index) # 训练集PBMC和测试集Malt相交的基因(20713)和蛋白质(10)
common_proteins = np.intersect1d(adata_protein.var.index, adata_protein_test.var.index)

In [ ]:
sciPENN = sciPENN_API([adata_gene], [adata_protein], adata_gene_test, train_batchkeys = ['donor'])

In [ ]:
start = time()
sciPENN.train(n_epochs = 200, ES_max = 12, decay_max = 6,
             decay_step = 0.1, lr = 10**(-3), weights_dir = "weights_dir/pbmc_to_malt", load=True)
imputed_test = sciPENN.predict()
time() - start

In [ ]:
embedding = sciPENN.embed()
embedding.write("sciPENN_maltembedding.h5ad")

In [ ]:
def corr2_coeff(A, B, pearson = True):
    if pearson:
        # Rowwise mean of input arrays & subtract from input arrays themeselves
        A_mA = A - A.mean(1)[:, None]
        B_mB = B - B.mean(1)[:, None]

        # Sum of squares across rows
        ssA = (A_mA**2).sum(1)
        ssB = (B_mB**2).sum(1)

        # Finally get corr coeff
        corr_mat = np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
        
        return corr_mat[range(corr_mat.shape[0]), range(corr_mat.shape[0])]
    
    else:
        corrs = [0.] * A.shape[0]
        
        for i in range(A.shape[0]):
            corrs[i] = spearmanr(A[i], B[i])[0]
            
        return corrs

In [ ]:
corrs = corr2_coeff(adata_protein_test.layers["imputed"].T, adata_protein_test.X.T)
corrs = pd.DataFrame(corrs)
corrs.index = adata_protein_test.var.index
corrs = corrs.dropna()
corrs

In [ ]:
corrs.mean()

In [ ]:
sq = lambda x, y: (x - y)**2

sqs = sq(adata_protein_test.layers["imputed"], adata_protein_test.X).mean(axis = 0)
sqs = pd.DataFrame(sqs)
sqs.index = adata_protein_test.var.index

In [ ]:
sqs.mean()

In [ ]:
r95 = (adata_protein_test.X < adata_protein_test.layers['q75'])
l95 = (adata_protein_test.X > adata_protein_test.layers['q25'])

print(f"Effective Coverage Probability for Nominal 50% PI: {(r95*l95).mean():.3f}")

In [ ]:
r95 = (adata_protein_test.X < adata_protein_test.layers['q90'])
l95 = (adata_protein_test.X > adata_protein_test.layers['q10'])

print(f"Effective Coverage Probability for Nominal 80% PI: {(r95*l95).mean():.3f}")